# 5.1.	데이터 수집하기 1 – 한 페이지 크롤링 하기

### 예제 5-1 selenium으로 다나와 검색 결과 URL 접속하기

In [1]:
from selenium import webdriver

browser = webdriver.Chrome('c:/playwithdata/chromedriver.exe')
url = "http://search.danawa.com/dsearch.php?query=무선청소기&tab=main"
browser.get(url)

### 예제 5-2 웹 페이지 HTML 정보 가져오기

In [3]:
from bs4 import BeautifulSoup

html = browser.page_source
soup = BeautifulSoup(html, 'html.parser')

### 예제 5-3 page1에 대한 무선청소기 정보 가져오기1

In [4]:
prod_items = soup.select('li.prod_item')
len(prod_items)

37

### 예제 5-4 page1에 대한 무선청소기 정보 가져오기2

In [5]:
prod_items = soup.select('ul.product_list > li.prod_item')
len(prod_items)

37

### 예제 5-5 page1에 대한 무선청소기 정보 가져오기3

In [8]:
prod_items = soup.select('div#productListArea > div.main_prodlist > ul.product_list > li.prod_item')
len(prod_items)

30

### 예제 5-6 상품명 정보 가져오기

In [9]:
title = prod_items[0].select('p.prod_name > a')[0].text.strip()
print(title)

샤오미 드리미 V9


### 예제 5-7 스펙 리스트 정보 가져오기

In [10]:
spec_list = prod_items[0].select('div.spec_list')[0].text.strip()
print(spec_list)

핸디/스틱청소기 / 핸디+스틱형 / 무선형 / 전압: 25.2V / 헤파필터 / 싸이클론 / 5단계여과 / 흡입력: 120AW / 충전시간: 3시간30분 / 사용시간: 60분 / 브러쉬: 바닥, 솔형, 틈새, 침구, 연장관 / 거치대 / 무게: 1.5kg / 색상:화이트 / 소비전력: 400W


### 예제 5-8 가격 정보 가져오기

In [11]:
price = prod_items[0].select('li.rank_one > p.price_sect > a > strong')[0].text.strip().replace(",","")
print(price)

137700


### 예제 5-9 반복문으로 검색결과 1페이지 상품 정보 추출하기

In [24]:
prod_data = []

for prod_item in prod_items:
    try: # ① 상품명 가져오기
        title = prod_item.select('p.prod_name > a')[0].text.strip()
    except:
        title = ''
    try: # ② 스펙 리스트 가져오기
        spec_list = prod_item.select('div.spec_list')[0].text.strip()
    except:
        spec_list = ''
    try: # ③ 가격 정보 가져오기 
        price = int(prod_item.select('li.rank_one > p.price_sect > a > strong')[0].text.strip().replace(",",""))
    except:
        price = 0
    prod_data.append([title, spec_list, price])

print(len(prod_data))
print(prod_data)

30
[['비엔필레 H-12', '핸디/스틱청소기 / 핸디형 / 유선형 / 헤파필터 / 브러쉬: 바닥, 솔형, 틈새, 연장관 / 셀프스탠딩 / 무게: 1.5kg / 색상:레드 / 소비전력: 400W / 크기(가로x세로x깊이): 280x380x200mm', 30010], ['좋은생활지웰 GWELL-502', '핸디/스틱청소기 / 핸디+스틱형 / 무선형 / 전압: 22.2V / 충전시간: 3~4시간 / 사용시간: 40분 / 배터리: 리튬이온 / 브러쉬: 바닥, 솔형, 틈새 / LED라이트 / 배터리잔량표시 / 색상:로즈골드 / 크기(가로x세로x깊이): 290x120x200mm', 78900], ['신일산업 SDC-B5000W', '물걸레청소기 / 스틱형 / 물걸레전용 / 회전식 / 무선형 / 전압: 16.8V / 충전시간: 3시간 / 사용시간: 40분 / 배터리: 리튬이온 / 물분사 / 셀프스탠딩 / 각도조절 / 무게: 3.9kg / 색상:레드 / 패드: 극세사(벨크로형) / 크기(가로x세로x깊이): 420x1150x190mm', 114750], ['이노템 아이언맨 IM-900', '물걸레청소기 / 스틱형 / 물걸레전용 / 왕복식 / 무선형 / 전압: 12.6V / 충전시간: 3시간 / 사용시간: 80분 / 배터리: 리튬이온 / 셀프스탠딩 / 가구손상방지 / 무게: 2.5~3kg / 소비전력: 30W / 색상:레드 / 패드: 극세사 / 크기(가로x세로x깊이): 240x1190x240mm', 54730], ['다이슨 V7 트리거 프로', '핸디/스틱청소기 / 핸디형 / 무선형 / 전압: 21.6V / 흡입력: 100AW / 충전시간: 3시간30분 / 사용시간: 30분 / 배터리: 리튬이온 / 무게: 1.71kg / 소비전력: 350W / 브러쉬: 미니모터헤드, 미니소프트더스팅, 스토번디트, 크레비스툴, 매트리스툴, 익스텐션호스 / 크기(가로x세로x깊이): 131x206x39mm', 419900], ['보쉬 GAS10.8V-LI', '핸디/스틱청소기 / 핸디형 / 무

### 예제 5-10 상품 정보 태그에서 원하는 정보 추출 함수

In [26]:
def get_prod_items(prod_items):
    prod_data = []
    for prod_item in prod_items:
        # ① 상품명 가져오기
        try:
            title = prod_item.select('p.prod_name > a')[0].text.strip()
        except:
            title = ''
        # ② 스펙 리스트 가져오기
        try: 
            spec_list = prod_item.select('div.spec_list')[0].text.strip()
        except:
            spec_list = ''
        # ③ 가격 정보 가져오기 
        try: 
            price = int(prod_item.select('li.rank_one > p.price_sect > a > strong')[0].text.strip().replace(",",""))
        except:
            price = 0
        prod_data.append([title, spec_list, price])
        
    return prod_data

### 예제 5-11 상품정보 가져오는 함수 호출 테스트

In [27]:
prod_items = soup.select('div#productListArea > div.main_prodlist > ul.product_list > li.prod_item')
prod_data = get_prod_items(prod_items)
print(len(prod_data))

30


# 5.2. 데이터 수집하기 2 - 여러 페이지 다나와 검색 페이지 크롤링하기

## 예제 5-12 다나와 검색 URL 만들어주는 함수

In [28]:
def get_search_page_url(keyword, page):
    return 'http://search.danawa.com/dsearch.php?query={}&volumeType=allvs&page={}&limit=30&sort=saveDESC&list=list&boost=true&addDelivery=N&tab=goods&tab=goods'.format(keyword, page)

keyword = '무선청소기'
page = 1
url = get_search_page_url(keyword, page)
print(url)

http://search.danawa.com/dsearch.php?query=무선청소기&volumeType=allvs&page=1&limit=30&sort=saveDESC&list=list&boost=true&addDelivery=N&tab=goods&tab=goods


## 예제 5-13 tqdm 설치 방법

In [19]:
!pip install tqdm

## 예제 5-14 tqdm 사용법

In [20]:
import time
from tqdm import tqdm_notebook

total_page = 10
for page in tqdm_notebook(range(1, total_page + 1)):
    time.sleep(5)

## 예제 5-15 실전 다나와 크롤링 해보기

In [29]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook

browser = webdriver.Chrome('c:/playwithdata/chromedriver.exe')
# 암묵적으로 웹 자원 로드를 위해 3초까지 기다려 준다.
browser.implicitly_wait(3)

keyword = '무선청소기'
total_page = 10
prod_data_total = []

# 진행정도를 잘표현해주는 tqdm을 적용 
for page in tqdm_notebook(range(1, total_page + 1)):
    # ① 검색 페이지 이동하기
    url = get_search_page_url(keyword, page)
    browser.get(url)
    # 페이지가 로딩완료되기 위한 시간을 5초를 준다. 
    time.sleep(5)
    
    # ② 현재 페이지 html 정보 가져오기
    html = browser.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # ③ 상품 정보 추출하기 
    prod_items = soup.select('div#productListArea > div.main_prodlist > ul.product_list > li.prod_item')
    prod_item_list = get_prod_items(prod_items)

    # ④ 추출 데이터 저장하기
    prod_data_total = prod_data_total + prod_item_list

In [30]:
print(len(prod_data_total))

300


## 예제 5-16 데이터 저장하기

In [31]:
import pandas as pd
data = pd.DataFrame(prod_data_total)
data.columns = ['상품명', '스펙 리스트', '가격']
data.to_excel('./files/3_1_danawa_crawling_result.xlsx', index=False)